In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.preprocessing import text, sequence
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Activation, Dropout, Dense
from sklearn.model_selection import train_test_split


In [2]:

train_df = pd.read_csv('text_emotion_train_val_set.csv')
train_df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [3]:
train_df = train_df.drop(['tweet_id', 'author'],axis=1)

In [4]:
train_df.head()


,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [6]:
train_df_1_anger = pd.read_csv('anger-ratings-0to1.test.gold.txt', error_bad_lines=False, sep='\t', header=None)
train_df_1_anger.columns = ['tweet_id', 'content', 'sentiment', 'sentiment_intensity']
train_df_1_anger.head()

,tweet_id,content,sentiment,sentiment_intensity
0,10941,At the point today where if someone says somet...,anger,0.319
1,10942,@CorningFootball IT'S GAME DAY!!!! T MIN...,anger,0.144
2,10943,This game has pissed me off more than any othe...,anger,0.898
3,10944,@spamvicious I've just found out it's Candice ...,anger,0.271
4,10945,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,anger,0.646


In [7]:
train_df_1_anger = train_df_1_anger.drop(['tweet_id'], axis=1)

In [8]:
train_df_1_anger.head()


,content,sentiment,sentiment_intensity
0,At the point today where if someone says somet...,anger,0.319
1,@CorningFootball IT'S GAME DAY!!!! T MIN...,anger,0.144
2,This game has pissed me off more than any othe...,anger,0.898
3,@spamvicious I've just found out it's Candice ...,anger,0.271
4,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,anger,0.646


In [9]:
train_df_1_anger = train_df_1_anger[train_df_1_anger.sentiment_intensity > 0.5]

In [10]:
train_df_1_anger = train_df_1_anger.drop(['sentiment_intensity'], axis=1)

In [11]:
train_df = [train_df, train_df_1_anger]

In [12]:
train_df = pd.concat(train_df)

In [13]:
train_df


,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm
4,@dannycastillo We want to trade with someone w...,neutral
5,Re-pinging @ghostridah14: why didn't you go to...,worry
6,"I should be sleep, but im not! thinking about ...",sadness
7,Hmmm. http://www.djhero.com/ is down,worry
8,@charviray Charlene my love. I miss you,sadness
9,@kelcouch I'm sorry at least it's Friday?,sadness


In [15]:
train_df_2_anger = pd.read_csv('anger-ratings-0to1.train.txt', error_bad_lines=False, sep='\t', header=None)
train_df_2_anger.columns = ['tweet_id', 'content', 'sentiment', 'sentiment_intensity']
train_df_2_anger = train_df_2_anger[train_df_2_anger.sentiment_intensity > 0.5]
train_df_2_anger = train_df_2_anger.drop(['tweet_id', 'sentiment_intensity'], axis=1)
train_df_2_anger.tail()

,content,sentiment
381,don't provoke me after letting me down !,anger
382,could never be a angry drunk lol yall weirdos ...,anger
383,"Yet we still have deaths, road rage, &amp; vio...",anger
396,I have a rage rage ep 2 coming out soon I'll k...,anger
505,Egyptian officials expressed frustration and o...,anger


In [17]:

train_df = [train_df, train_df_2_anger]
train_df = pd.concat(train_df)
train_df_1_fear = pd.read_csv('fear-ratings-0to1.test.gold.txt', error_bad_lines=False, sep='\t', header=None)
train_df_2_fear = pd.read_csv('fear-ratings-0to1.train.txt', error_bad_lines=False, sep='\t', header=None)
train_df_fear = [train_df_1_fear, train_df_2_fear]
train_df_fear = pd.concat(train_df_fear)
train_df_fear.columns = ['tweet_id', 'content', 'sentiment', 'sentiment_intensity']
train_df_fear = train_df_fear[train_df_fear.sentiment_intensity > 0.5]
train_df_fear = train_df_fear.drop(['tweet_id', 'sentiment_intensity'], axis=1)
train_df_fear.tail()
train_df_fear.shape

(1000, 2)

In [18]:
train_df = [train_df, train_df_fear]
train_df = pd.concat(train_df)

In [19]:

print(train_df.head())
train_df = train_df.sample(frac=1).reset_index(drop=True)
print(len(train_df))
train_df.head()

                                             content   sentiment
0  @tiffanylue i know  i was listenin to bad habi...       empty
1  Layin n bed with a headache  ughhhh...waitin o...     sadness
2                Funeral ceremony...gloomy friday...     sadness
3               wants to hang out with friends SOON!  enthusiasm
4  @dannycastillo We want to trade with someone w...     neutral
42098


,content,sentiment
0,Youtube's tubes appear to be clogging up somew...,worry
1,@TomFelton d'aaaaawhh you're so cute,love
2,My aim is not working due to network problems,worry
3,Allergies suck ducks nuts. &lt;=====8@8===...,sadness
4,@kyoisorange ok ermm I'll meet you in the gym ...,enthusiasm


In [20]:

y_pandas_df = pd.get_dummies(train_df['sentiment'])
print(type(y_pandas_df))
print(y_pandas_df.shape)
print(y_pandas_df.ndim)
y_pandas_df.head()

<class 'pandas.core.frame.DataFrame'>
(42098, 14)
2


,anger,boredom,empty,enthusiasm,fear,fun,happiness,hate,love,neutral,relief,sadness,surprise,worry
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [21]:
y = y_pandas_df.values
print("properties of y")
print("type : {}, dimensions : {}, shape : {}, total no. of elements : {}, data type of each element: {}, size of each element {} bytes".format(type(y), y.ndim, y.shape, y.size, y.dtype, y.itemsize))

properties of y
type : <class 'numpy.ndarray'>, dimensions : 2, shape : (42098, 14), total no. of elements : 589372, data type of each element: uint8, size of each element 1 bytes


In [22]:
train_df = train_df.drop(['sentiment'], axis=1)

In [23]:

train_df.shape
x = train_df.values.flatten()

In [24]:
print(x)
print("properties of x")
print("type : {}, dimensions : {}, shape : {}, total no. of elements : {}, data type of each element: {}, size of each element {} bytes".format(type(x), x.ndim, x.shape, x.size, x.dtype, x.itemsize))

[ "Youtube's tubes appear to be clogging up somewhat today, connection errors a plenty."
 "@TomFelton d'aaaaawhh you're so cute"
 'My aim is not working due to network problems' ...,
 'waking up again, this time with a hot cup of coffee'
 'FUCK wasting deodorant... i aint goin no where... i aint havin company  let me funk it to ya!'
 "Soon we are going to look at a cabaret, it's going to be fun!"]
properties of x
type : <class 'numpy.ndarray'>, dimensions : 1, shape : (42098,), total no. of elements : 42098, data type of each element: object, size of each element 8 bytes


In [32]:
list_of_classes = ['anger', 'boredom', 'empty', 'enthusiasm', 'fear', 'fun', 'happiness', 'hate', 'love', 'neutral', 'relief', 'sadness', 'surprise', 'worry']
max_features = 20000
max_text_length = 400
embedding_dims = 50
batch_size = 32
epochs = 3
num_filters_1 = 250
num_filters_2 = 250
filter_size = 3

In [30]:
x_tokenizer = text.Tokenizer(num_words=max_features)
x_tokenizer.fit_on_texts(list(x))
x_tokenized = x_tokenizer.texts_to_sequences(x)
x_train_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)

In [27]:
model = Sequential()

model.add(Embedding(max_features, embedding_dims, input_length=max_text_length))
model.add(Dropout(0.2))

model.add(Conv1D(filters=num_filters_1, kernel_size=filter_size, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(num_filters_2))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(14))
model.add(Activation('sigmoid'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [28]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
act

In [33]:
model.fit(x_train_val, y, batch_size=batch_size, epochs=epochs)

Epoch 1/3
42098/42098 [==============================]42098/42098 [==============================] - 365s 9ms/step - loss: 0.2282 - acc: 0.9286

Epoch 2/3
42098/42098 [==============================]42098/42098 [==============================] - 385s 9ms/step - loss: 0.2143 - acc: 0.9289

Epoch 3/3
42098/42098 [==============================]42098/42098 [==============================] - 420s 10ms/step - loss: 0.2017 - acc: 0.9299



In [34]:

model.save('emo_class_keras_model.h5')

In [35]:

from tensorflow.python.keras.models import load_model

def predict(test_set):
    global x_tokenizer
    model2 = load_model('emo_class_keras_model.h5')
    test_df = pd.read_csv(test_set)
    x_test = test_df['comment'].values
    x_test_tokenized = x_tokenizer.texts_to_sequences(x_test)
    x_testing = sequence.pad_sequences(x_test_tokenized, maxlen=max_text_length)
    y_testing = model2.predict(x_testing, verbose = 1)
    sample_submission = pd.read_csv("submission.csv")
    sample_submission[list_of_classes] = y_testing
    sample_submission.to_csv("solution.csv", index=False)

In [36]:
print(predict('text_emotion_classification_test_set.csv'))

19/19 [==============================]19/19 [==============================] - 0s 4ms/step

None
